In [0]:
%pip install nbformat databricks-sdk[openai]==0.38.0
dbutils.library.restartPython()

In [0]:
%run ../prompts/prompts_notebook_sql

In [0]:
import base64

import databricks.sdk
from databricks.sdk.service.workspace import ImportFormat
import nbformat
import numpy as np 

In [0]:
candidate_dict = [
  ("X","/Workspace/Users/vibhor.nigam@databricks.com/interview-scripts/interview-grading/example-notebooks-coding-style/03-Coding-Style-X"),
]
human_graded_dict = {
  "X": [1,0,1,1,1,1,1,0]
}

w = databricks.sdk.WorkspaceClient()

In [0]:
def get_code_from_notebooks(workspace_client, notebook_path):
   notebook = workspace_client.workspace.export(notebook_path, 
                                    format=ImportFormat.JUPYTER)
   ipynb = base64.decodebytes(notebook.content.encode('ascii')).decode("utf-8")
   notebook = nbformat.reads(ipynb, as_version=4)
   code_dict = {}
   x=0
   for cell in notebook.cells:
      if cell.cell_type == 'code':
         code_dict[x] = cell.source
         x+=1
   context = ''.join(code_dict.values())
   return context
# ipynb

In [0]:
import json
def clean_up_response(content):
  char_list = ["`","```json", "\n", "json"]
  for i in char_list:
    content=content.replace(i, "")
  content_json = json.loads(content)
  print(content_json)
  return content_json

In [0]:
def evaluate_model(diff):
  unmatched_elements = [i for i in diff if i != 0]
  diff_in_score = sum(unmatched_elements)
  return diff_in_score

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import ChatMessage, ChatMessageRole
import pandas as pd

def evaluate_responses(w, context, prompt_dict):
    prompt_list = [f"""{prompt_dict[f'prompt_instructions_{i+1}']} ```{context}``` """ for i in range(0,len(prompt_dict))]
    answer_list = []
    
    for prompt in prompt_list:
        response = w.serving_endpoints.query(
            name="databricks-meta-llama-3-70b-instruct",
            messages=[
                ChatMessage(
                    role=ChatMessageRole.SYSTEM, content=system_prompt
                ),
                ChatMessage(
                    role=ChatMessageRole.USER, content=prompt
                ),
            ],
            max_tokens=128,
            
        )
        curr_response = response.choices[0].message.content
        # print(curr_response)
        curr_response_json = clean_up_response(curr_response)
        answer_list.append(curr_response_json)
    answer_list_df = pd.DataFrame(answer_list)

    return answer_list_df

In [0]:
# spark.sql(f" drop table if exists users.abhay_jalisatgi.gen_ai_eval ")

In [0]:
error_dict = {}
answers_dict = {}
for candidate in candidate_dict:
  context = get_code_from_notebooks(w, candidate[1])
  answers = evaluate_responses(w, context, coding_prompt_dict)
  answers['candidate'] = candidate[0]
  answers['score'] = answers['score'].astype('float64')
  answers_dict[candidate[0]] = answers

  #  -- uncomment below when writing actually to the table 
  # spark.createDataFrame(answers).write.mode("append").saveAsTable("users.abhay_jalisatgi.gen_ai_eval")

  # human_answers_list = human_graded_dict[candidate[0]]
  # model_asnwers_list = answers['score'].tolist()
  # diff = [human_answers_list[i] - model_asnwers_list[i] for i in range(len(human_answers_list))]
  
  # abs_error = np.abs(diff).sum()
  # inaccurate_predictions = len([x for x in diff if x != 0])

  # error_dict[candidate[0]] = {
  #   'abs_error': abs_error, 
  #   'inaccurate_predictions': inaccurate_predictions, 
  #   'human_answers': human_answers_list,
  #   'model_answers': model_asnwers_list,
  #   'diff': diff
  #   }

In [0]:
answers_dict['X'].display()

In [0]:
error_dict

In [0]:
# prompt_instructions_6 = sql_prompt_dict['prompt_instructions_6']
# prompt6 = f"""{prompt_instructions_6} ```{context_dict[candidate]}``` """

In [0]:
# import json
# from databricks.sdk import WorkspaceClient

# w = WorkspaceClient()
# openai_client = w.serving_endpoints.get_open_ai_client()

# response_format = {
#   "type": "json_schema",
#   "json_schema":{
#     "name": "results",
#     "schema": {
#       "type": "object",
#       "properties": {
#         "score": { "type": "integer" },
#         "explanation": { "type": "string" }
#       }
#     }
#   }
# }

# response = openai_client.chat.completions.create(
#     model="databricks-dbrx-instruct",
#     messages=[
#       {
#         "role": "system",
#         "content": "You are a helpful assistant."
#       },
#       {
#         "role": "user",
#         "content": prompt1,
#       }
#     ],
#     response_format=response_format,
#     max_tokens=256
# )
# print(json.dumps(response.choices[0].message.model_dump(), indent=2))